In [2]:
import streamlit as st
import chromadb
import google.generativeai as genai



c:\Users\ahmed\anaconda3\envs\education\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

genai.configure(api_key="")

In [4]:
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def extract_and_chunk_pdf(file):
    reader = PdfReader(file)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(text)
    return chunks

 # run this function on all files that ends with .pdf on the directory


In [5]:
import os

def extract_and_chunk_pdfs_from_dir(directory_path):
    all_chunks = []
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "rb") as file:
                chunks = extract_and_chunk_pdf(file)
                all_chunks.extend(chunks)
    
    return all_chunks


In [6]:
b=extract_and_chunk_pdfs_from_dir(r"./pdf") 

In [7]:
documents= b

In [8]:
from chromadb import EmbeddingFunction
from google.generativeai import embed_content

class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    def __call__(self, input: list[str]) -> list[list[float]]:
        task = "retrieval_document" if self.document_mode else "retrieval_query"
        embeddings = []

        for text in input:
            try:
                response = embed_content(
                    model="models/embedding-001",
                    content=text,
                    task_type=task,
                )
                embeddings.append(response["embedding"])
            except Exception as e:
                print(f"Error embedding: {e}")
                embeddings.append([0.0] * 768)  # fallback vector

        return embeddings


In [9]:
import chromadb

DB_NAME = "white_papers_db"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True


try:
	chroma_client = chromadb.PersistentClient(path="./chroma_db")  
	db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)
	print(f"Collection '{DB_NAME}' initialized successfully.")
except Exception as e:
	print(f"Error initializing database or collection: {e}")
	raise

# Add documents to the collection
try:
	db.add(documents=documents, ids=[str(i) for i in range(len(documents))])
	print("Documents added successfully.")
except Exception as e:
	print(f"Error adding documents to the collection: {e}")
	raise

C:\Users\ahmed\AppData\Local\Temp\ipykernel_15296\3056851207.py:5: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embed_fn = GeminiEmbeddingFunction()


Collection 'white_papers_db' initialized successfully.
Documents added successfully.


In [10]:
db.name

'white_papers_db'

In [ ]:
# get db name 

db.count()  # Check the number of documents in the collection
db.peek(3)

NameError: name 'db' is not defined

In [53]:
from IPython.display import Markdown

embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "hi"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Keeping. Would you like me to walk you through the settings menu instead?"
General Knowledge Agent
The General Knowledge Agent is specialized in answering factual questions about the world, 
history, science, culture, and other general topics. This agent accesses a broad knowledge 
base to answer factual questions, provides biographical information about people, offers 
contextual explanations of concepts and phenomena, maintains grounding in factual

In [54]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative and conversational bot for university students that answers questions and explain concepts using text from the reference passage included below. 
you can use other information from your own knowledge only if it is necessary to ensure the answer is correct and complete.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone and act like human. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative and conversational bot for university students that answers questions and explain concepts using text from the reference passage included below. 
you can use other information from your own knowledge only if it is necessary to ensure the answer is correct and complete.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone and act like human. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: hi
PASSAGE: Keeping. Would you like me to walk you through the settings menu instead?" General Knowledge Agent The General Knowledge Agent is specialized in answering factual questions about the world,  history, science, culture, and other general topics. This agent accesses a broad knowledge  base to answer factual questions, provides biogra

In [55]:
model = genai.GenerativeModel("gemini-2.0-flash")

# Generate content
response = model.generate_content(prompt)

# Display the response as Markdown
Markdown(response.text)

Hi there! How can I help you today? I'm here to answer your questions and explain concepts in a way that's easy to understand. Just let me know what you're curious about!
